In [1]:
import sys
import platform
from pyspark.sql import SparkSession
import pyspark

spark = (
    SparkSession.builder
    .appName('BDA-Project')
    .config('spark.sql.session.timeZone', 'UTC')
    .config('spark.sql.shuffle.partitions', '4')
    .getOrCreate()
)

spark.sparkContext.setLogLevel('WARN')

print(f'Spark version: {spark.version}')
print(f'PySpark version: {pyspark.__version__}')
print(f'Python version: {sys.version.split()[0]}')
print(f'Session timezone: {spark.conf.get("spark.sql.session.timeZone")}')
print(f'Shuffle partitions: {spark.conf.get("spark.sql.shuffle.partitions")}')

Spark version: 4.0.1
PySpark version: 4.0.1
Python version: 3.10.19
Session timezone: UTC
Shuffle partitions: 4


In [3]:
import os
import pandas as pd
from datetime import datetime
import random
import numpy as np
from pathlib import Path

print("\n" + "="*70)
print("PHASE 1: BLOCKCHAIN + PRICES INTEGRATION - FULL SPARK")
print("="*70)

print("\n📖 Etape 1: Chargement des donnees de prix...")
df_prices_pd = pd.read_csv("data/prices/btc_1h_data_2018_to_2025.csv")
print(f"✓ {len(df_prices_pd)} heures chargees")

print("\n🔗 Etape 2: Preparation des donnees blockchain...")

blockchain_real_parquet = "data/parquet/blockchain_real_hourly.parquet"

if os.path.exists(blockchain_real_parquet):
    print("✅ DONNEES REELLES TROUVEES - Utilisation des vraies donnees blockchain!")
    df_hourly_blockchain = pd.read_parquet(blockchain_real_parquet)
    df_hourly_blockchain['timestamp'] = pd.to_datetime(df_hourly_blockchain['timestamp'])
    print(f"   ✓ {len(df_hourly_blockchain)} heures de donnees reelles chargees")
    print(f"   ✓ Periode: {df_hourly_blockchain['timestamp'].min()} a {df_hourly_blockchain['timestamp'].max()}")
    is_real_blockchain = True
else:
    print("⚠️  Donnees blockchain reelles non trouvees - Generation synthetiques de fallback...")
    
    transactions = []

    for idx, row in df_prices_pd.iterrows():
        try:
            timestamp_str = row['Open time']
            close_price = float(row['Close'])
            volume = float(row['Volume']) if pd.notna(row['Volume']) else 0
            
            if pd.api.types.is_numeric_dtype(timestamp_str):
                timestamp = int(timestamp_str)
            else:
                try:
                    dt = datetime.fromisoformat(str(timestamp_str).replace('Z', '+00:00'))
                    timestamp = int(dt.timestamp())
                except:
                    dt = pd.to_datetime(timestamp_str)
                    timestamp = int(dt.timestamp())
            
            base_tx_count = int(100 + (volume / 100000)) if volume > 0 else 100
            tx_count = max(50, min(500, base_tx_count + random.randint(-50, 50)))
            
            for tx_idx in range(tx_count):
                tx_timestamp = timestamp + random.randint(0, 3599)
                avg_amount_btc = max(0.01, 10 / (close_price / 1000)) if close_price > 0 else 0.5
                amount_btc = avg_amount_btc * random.uniform(0.5, 2.0)
                amount_satoshi = int(amount_btc * 100_000_000)
                
                tx = {
                    'tx_hash': f"tx_{idx}_{tx_idx}",
                    'timestamp': tx_timestamp,
                    'input_count': random.randint(1, 5),
                    'output_count': random.randint(1, 5),
                    'total_output_value': amount_satoshi,
                    'block_hash': f"block_{idx}"
                }
                transactions.append(tx)
            
            if (idx + 1) % 500 == 0:
                print(f"  ✓ {idx + 1}/{len(df_prices_pd)} heures...")
        except Exception as e:
            continue

    print(f"✓ {len(transactions)} transactions synthetiques generees")

    df_txs = pd.DataFrame(transactions)
    df_txs['hour_dt'] = pd.to_datetime(df_txs['timestamp'], unit='s').dt.floor('h')

    df_hourly_blockchain = df_txs.groupby('hour_dt').agg({
        'tx_hash': 'count',
        'input_count': 'mean',
        'output_count': 'mean',
        'total_output_value': ['sum', 'mean', 'std']
    }).reset_index()

    df_hourly_blockchain.columns = [
        'timestamp', 'tx_count', 'avg_inputs', 'avg_outputs',
        'total_volume_satoshi', 'avg_amount_satoshi', 'std_amount_satoshi'
    ]

    for col_name in ['total_volume_satoshi', 'avg_amount_satoshi', 'std_amount_satoshi']:
        df_hourly_blockchain[col_name] = df_hourly_blockchain[col_name] / 100_000_000

    df_hourly_blockchain = df_hourly_blockchain.rename(columns={
        'total_volume_satoshi': 'total_volume_btc',
        'avg_amount_satoshi': 'avg_amount_btc',
        'std_amount_satoshi': 'std_amount_btc'
    })

    print(f"✓ {len(df_hourly_blockchain)} heures aggregees")
    is_real_blockchain = False

print("\n💾 Etape 3: Standardisation et sauvegarde en Parquet...")

df_hourly_blockchain['timestamp'] = df_hourly_blockchain['timestamp'].astype(str)

required_cols = ['timestamp', 'tx_count', 'avg_inputs', 'avg_outputs', 'total_volume_btc', 'avg_amount_btc', 'std_amount_btc']
df_hourly_blockchain = df_hourly_blockchain[required_cols]

os.makedirs("data/parquet", exist_ok=True)
blockchain_parquet = "data/parquet/blockchain_hourly.parquet"
df_hourly_blockchain.to_parquet(blockchain_parquet)
print(f"✓ Blockchain sauvegardee: {blockchain_parquet}")
print(f"   Type: {'REELLES' if is_real_blockchain else 'SYNTHETIQUES (fallback)'}")

print("\n⚡ Etape 4: Chargement des donnees dans Spark...")
df_prices_spark = spark.read.csv("data/prices/btc_1h_data_2018_to_2025.csv", header=True, inferSchema=True)
print(f"✓ Prices charges dans Spark: {df_prices_spark.count()} rows")

print("\n📊 Etape 5: Chargement blockchain dans Spark...")
df_blockchain_spark = spark.read.parquet(blockchain_parquet)
print(f"✓ Blockchain charge dans Spark: {df_blockchain_spark.count()} rows")
print(f"   Source: {'DONNEES REELLES' if is_real_blockchain else 'donnees synthetiques'}")

print("\n🔄 Etape 6: JOIN blockchain + prices en Spark SQL...")

from pyspark.sql.functions import col, to_timestamp, date_trunc

df_prices_spark = df_prices_spark.withColumn(
    "hour_ts", 
    date_trunc('hour', to_timestamp(col("Open time")))
)

df_blockchain_spark = df_blockchain_spark.withColumn(
    "hour_ts", 
    to_timestamp(col("timestamp"))
)

df_prices_spark.createOrReplaceTempView("prices")
df_blockchain_spark.createOrReplaceTempView("blockchain")

df_integrated = spark.sql("""
    SELECT 
        p.*,
        b.tx_count,
        b.avg_inputs,
        b.avg_outputs,
        b.total_volume_btc,
        b.avg_amount_btc,
        b.std_amount_btc
    FROM prices p
    LEFT JOIN blockchain b
    ON p.hour_ts = b.hour_ts
""")

print(f"✓ JOIN complete: {df_integrated.count()} rows")

print("\n💾 Etape 7: Cache du DataFrame integre...")
df_integrated.cache()

print(f"✓ Spark DataFrame final cree et cache!")
print(f"  Nombre de lignes: {df_integrated.count()}")
print(f"  Nombre de colonnes: {len(df_integrated.columns)}")

print(f"\n✅ PHASE 1 COMPLETEE - FULL SPARK!")
print(f"   df_integrated: DataFrame Spark avec blockchain + prices")
if is_real_blockchain:
    print(f"   UTILISE DONNEES REELLES extraites des fichiers .dat")
else:
    print(f"   (Utilise donnees synthetiques - pour ameliorer: executez BONUS PHASE)")


PHASE 1: BLOCKCHAIN + PRICES INTEGRATION - FULL SPARK

📖 Etape 1: Chargement des donnees de prix...
✓ 68543 heures chargees

🔗 Etape 2: Preparation des donnees blockchain...
✅ DONNEES REELLES TROUVEES - Utilisation des vraies donnees blockchain!
   ✓ 1888 heures de donnees reelles chargees
   ✓ Periode: 2012-06-09 22:00:00+00:00 a 2012-08-30 09:00:00+00:00

💾 Etape 3: Standardisation et sauvegarde en Parquet...
✓ Blockchain sauvegardee: data/parquet/blockchain_hourly.parquet
   Type: REELLES

⚡ Etape 4: Chargement des donnees dans Spark...
✓ Prices charges dans Spark: 68543 rows

📊 Etape 5: Chargement blockchain dans Spark...
✓ Blockchain charge dans Spark: 1888 rows
   Source: DONNEES REELLES

🔄 Etape 6: JOIN blockchain + prices en Spark SQL...
✓ JOIN complete: 68543 rows

💾 Etape 7: Cache du DataFrame integre...
✓ Spark DataFrame final cree et cache!
  Nombre de lignes: 68543
  Nombre de colonnes: 19

✅ PHASE 1 COMPLETEE - FULL SPARK!
   df_integrated: DataFrame Spark avec blockchai

In [4]:
from pyspark.sql.functions import lead, when, col
from pyspark.sql.window import Window

print("\n" + "="*70)
print("PHASE 2: DATA CLEANING - Using Integrated Data")
print("="*70)

df_clean = df_integrated.filter(col("Open time").isNotNull())
df_clean = df_clean.sort("Open time")

print(f"\n✓ Integrated dataset loaded: {df_clean.count()} rows")
print(f"✓ Columns: {len(df_clean.columns)}")

window_spec = Window.orderBy("Open time")
df_clean = df_clean.withColumn(
    "next_close",
    lead("Close").over(window_spec)
)

df_clean = df_clean.withColumn(
    "price_direction",
    when(col("next_close") > col("Close"), 1).otherwise(0)
)

df_clean = df_clean.filter(col("price_direction").isNotNull())

print(f"\n✓ Data cleaned!")
print(f"✓ Final shape: {df_clean.count()} rows")

print(f"\n=== First 3 rows with target ===")
df_clean.select("Open time", "Close", "next_close", "price_direction").show(3)

print(f"\n=== Blockchain columns available ===")
df_clean.select("Open time", "tx_count", "avg_inputs", "total_volume_btc", "price_direction").show(3)


PHASE 2: DATA CLEANING - Using Integrated Data

✓ Integrated dataset loaded: 68542 rows
✓ Columns: 19

✓ Data cleaned!
✓ Final shape: 68542 rows

=== First 3 rows with target ===
+-------------------+--------+----------+---------------+
|          Open time|   Close|next_close|price_direction|
+-------------------+--------+----------+---------------+
|2018-01-01 00:00:00|13529.01|  13203.06|              0|
|2018-01-01 01:00:00|13203.06|  13330.18|              1|
|2018-01-01 02:00:00|13330.18|  13410.03|              1|
+-------------------+--------+----------+---------------+
only showing top 3 rows

=== Blockchain columns available ===
+-------------------+--------+----------+----------------+---------------+
|          Open time|tx_count|avg_inputs|total_volume_btc|price_direction|
+-------------------+--------+----------+----------------+---------------+
|2018-01-01 00:00:00|    NULL|      NULL|            NULL|              0|
|2018-01-01 01:00:00|    NULL|      NULL|           

In [ ]:
# Empty cell for organization

In [5]:
from pyspark.sql.functions import (
    col, lag, avg, stddev, log, 
    abs as spark_abs, round as spark_round, coalesce
)
from pyspark.sql.window import Window

print("\n" + "="*70)
print("PHASE 3: FEATURE ENGINEERING - Price + Blockchain")
print("="*70)

window_24h = Window.orderBy("Open time").rowsBetween(-24, 0)
window_7d = Window.orderBy("Open time").rowsBetween(-168, 0)

df_features = df_clean.filter(col("Open time").isNotNull())

print("\n📊 Creating Price Features...")

df_features = df_features.withColumn(
    "hourly_return",
    spark_round(((col("Close") - col("Open")) / col("Open")) * 100, 4)
)

df_features = df_features.withColumn(
    "hl_range",
    spark_round((col("High") - col("Low")) / col("Open") * 100, 4)
)

df_features = df_features.withColumn(
    "close_ma_24h",
    spark_round(avg("Close").over(window_24h), 2)
)

df_features = df_features.withColumn(
    "close_ma_7d",
    spark_round(avg("Close").over(window_7d), 2)
)

df_features = df_features.withColumn(
    "volatility_24h",
    spark_round(stddev("Close").over(window_24h), 2)
)

df_features = df_features.withColumn(
    "volume_ma_24h",
    spark_round(avg("Volume").over(window_24h), 2)
)

df_features = df_features.withColumn(
    "volume_ratio",
    spark_round(col("Volume") / col("volume_ma_24h"), 4)
)

print("\n⛓️  Creating Blockchain Features...")

df_features = df_features.withColumn(
    "tx_count_ma_24h",
    spark_round(avg("tx_count").over(window_24h), 2)
)

df_features = df_features.withColumn(
    "tx_count_ratio",
    spark_round(col("tx_count") / col("tx_count_ma_24h"), 4)
)

df_features = df_features.withColumn(
    "volume_btc_ma_24h",
    spark_round(avg("total_volume_btc").over(window_24h), 4)
)

df_features = df_features.withColumn(
    "volume_btc_ratio",
    spark_round(col("total_volume_btc") / col("volume_btc_ma_24h"), 4)
)

df_features = df_features.withColumn(
    "avg_inputs_log",
    spark_round(log(col("avg_inputs") + 1), 4)
)

df_features = df_features.withColumn(
    "avg_outputs_log",
    spark_round(log(col("avg_outputs") + 1), 4)
)

df_features = df_features.withColumn(
    "price_to_onchain",
    spark_round(col("Close") / (col("total_volume_btc") + 1), 4)
)

df_features = df_features.withColumn(
    "io_ratio",
    spark_round(col("avg_inputs") / (col("avg_outputs") + 0.1), 4)
)

print(f"\n✓ Features engineered!")
print(f"✓ Shape: {df_features.count()} rows")

print(f"\n📋 === PRICE FEATURES ===")
df_features.select(
    "Open time", "hourly_return", "hl_range", "close_ma_24h", "volatility_24h", "volume_ratio"
).show(3, truncate=False)

print(f"\n⛓️  === BLOCKCHAIN FEATURES ===")
df_features.select(
    "Open time", "tx_count", "tx_count_ratio", "total_volume_btc", "volume_btc_ratio", "avg_inputs"
).show(3, truncate=False)

print(f"\n✅ FEATURES CREATED")


PHASE 3: FEATURE ENGINEERING - Price + Blockchain

📊 Creating Price Features...

⛓️  Creating Blockchain Features...

✓ Features engineered!
✓ Shape: 68542 rows

📋 === PRICE FEATURES ===
+-------------------+-------------+--------+------------+--------------+------------+
|Open time          |hourly_return|hl_range|close_ma_24h|volatility_24h|volume_ratio|
+-------------------+-------------+--------+------------+--------------+------------+
|2018-01-01 00:00:00|-1.3608      |2.3013  |13529.01    |NULL          |1.0         |
|2018-01-01 01:00:00|-2.4091      |3.256   |13366.04    |230.48        |0.9279      |
|2018-01-01 02:00:00|0.9633       |1.6544  |13354.08    |164.28        |1.0247      |
+-------------------+-------------+--------+------------+--------------+------------+
only showing top 3 rows

⛓️  === BLOCKCHAIN FEATURES ===
+-------------------+--------+--------------+----------------+----------------+----------+
|Open time          |tx_count|tx_count_ratio|total_volume_btc|

In [6]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, Imputer
from pyspark.sql.functions import col, year, expr, randn, abs as spark_abs

print("\n" + "="*70)
print("PHASE 4: ML DATA PREPARATION")
print("="*70)

feature_cols = [
    'hourly_return', 'hl_range', 'close_ma_24h', 'close_ma_7d', 'volatility_24h', 'volume_ratio',
    'tx_count', 'tx_count_ratio', 'total_volume_btc', 'volume_btc_ratio',
    'avg_inputs_log', 'avg_outputs_log', 'price_to_onchain', 'io_ratio'
]

print(f"\nFeatures for ML ({len(feature_cols)} total):")
for i, feat in enumerate(feature_cols, 1):
    print(f"   {i:2d}. {feat}")

print(f"\nChecking temporal overlap...")
print(f"   Input: {df_features.count()} rows")

df_features_filtered = df_features.filter(col('tx_count').isNotNull())
print(f"   Rows with blockchain: {df_features_filtered.count()}")

if df_features_filtered.count() > 0:
    df_ml = df_features_filtered
    print(f"   Using REAL blockchain data")
else:
    print(f"\n   WARNING: NO OVERLAP - Using synthetic fallback")
    df_ml = df_features.filter(col('price_direction').isNotNull())
    df_ml = df_ml.withColumn('tx_count', spark_abs(expr('cast(randn(42) * 500 + 1000 as double)')))
    df_ml = df_ml.withColumn('tx_count_ratio', expr('tx_count / 1000.0'))
    df_ml = df_ml.withColumn('total_volume_btc', spark_abs(expr('randn(43) * 100 + 200')))
    df_ml = df_ml.withColumn('volume_btc_ratio', expr('total_volume_btc / 200.0'))
    df_ml = df_ml.withColumn('avg_inputs_log', expr('log1p(abs(randn(44) * 1 + 2))'))
    df_ml = df_ml.withColumn('avg_outputs_log', expr('log1p(abs(randn(45) * 1 + 2))'))
    df_ml = df_ml.withColumn('price_to_onchain', expr('Close / (total_volume_btc + 1)'))
    df_ml = df_ml.withColumn('io_ratio', expr('exp(avg_inputs_log) / (exp(avg_outputs_log) + 0.1)'))
    print(f"   Synthetic features generated for {df_ml.count()} rows")

print(f"\nImputing missing values...")
imputer = Imputer(inputCols=feature_cols, outputCols=feature_cols, strategy='median')
df_ml = imputer.fit(df_ml).transform(df_ml)
df_ml = df_ml.filter(col('price_direction').isNotNull())

print(f"\nAssembling features...")
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features_raw', handleInvalid='skip')
df_ml = assembler.transform(df_ml)

print(f"\nScaling features...")
scaler = StandardScaler(inputCol='features_raw', outputCol='features', withMean=True, withStd=True)
df_ml = scaler.fit(df_ml).transform(df_ml)

print(f"\nTemporal Train/Test Split...")
df_ml = df_ml.withColumn('year', year('Open time'))
df_train = df_ml.filter(col('year') < 2024)
df_test = df_ml.filter(col('year') >= 2024)

print(f"\nTrain: {df_train.count()} rows (2018-2023)")
print(f"Test: {df_test.count()} rows (2024-2025)")
print(f"\n✅ PHASE 4 COMPLETE!")


PHASE 4: ML DATA PREPARATION

Features for ML (14 total):
    1. hourly_return
    2. hl_range
    3. close_ma_24h
    4. close_ma_7d
    5. volatility_24h
    6. volume_ratio
    7. tx_count
    8. tx_count_ratio
    9. total_volume_btc
   10. volume_btc_ratio
   11. avg_inputs_log
   12. avg_outputs_log
   13. price_to_onchain
   14. io_ratio

Checking temporal overlap...
   Input: 68542 rows
   Rows with blockchain: 0

   Synthetic features generated for 68542 rows

Imputing missing values...

Assembling features...

Scaling features...

Temporal Train/Test Split...

Train: 52464 rows (2018-2023)
Test: 16078 rows (2024-2025)

✅ PHASE 4 COMPLETE!


In [8]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

print("\n" + "="*70)
print("PHASE 5: MODEL TRAINING")
print("="*70)

print("\nTraining Logistic Regression...")
lr = LogisticRegression(labelCol='price_direction', featuresCol='features', maxIter=100, regParam=0.01)
model_lr = lr.fit(df_train)
pred_lr = model_lr.transform(df_test)

print("\nTraining Random Forest...")
rf = RandomForestClassifier(labelCol='price_direction', featuresCol='features', numTrees=50, maxDepth=10, seed=42)
model_rf = rf.fit(df_train)
pred_rf = model_rf.transform(df_test)

evaluator_auc = BinaryClassificationEvaluator(labelCol='price_direction', rawPredictionCol='rawPrediction', metricName='areaUnderROC')
evaluator_acc = MulticlassClassificationEvaluator(labelCol='price_direction', predictionCol='prediction', metricName='accuracy')

auc_lr = evaluator_auc.evaluate(pred_lr)
acc_lr = evaluator_acc.evaluate(pred_lr)
auc_rf = evaluator_auc.evaluate(pred_rf)
acc_rf = evaluator_acc.evaluate(pred_rf)

print("\n" + "="*60)
print("MODEL COMPARISON (with Price + Blockchain Features)")
print("="*60)
print(f"{'Model':<20} {'AUC':<15} {'Accuracy':<15}")
print("-"*60)
print(f"{'Logistic Reg':<20} {auc_lr:<15.4f} {acc_lr:<15.4f}")
print(f"{'Random Forest':<20} {auc_rf:<15.4f} {acc_rf:<15.4f}")

best_auc = max(auc_lr, auc_rf)
best_model = "Random Forest" if auc_rf == best_auc else "Logistic Regression"
print(f"\nBest Model (AUC): {best_model} ({best_auc:.4f})")

print(f"\n✅ TRAINING COMPLETE!")


PHASE 5: MODEL TRAINING

Training Logistic Regression...

Training Random Forest...

MODEL COMPARISON (with Price + Blockchain Features)
Model                AUC             Accuracy       
------------------------------------------------------------
Logistic Reg         0.5313          0.5140         
Random Forest        0.5155          0.5096         

Best Model (AUC): Logistic Regression (0.5313)

✅ TRAINING COMPLETE!


In [ ]:
# Empty cell for organization

In [7]:
import struct
import hashlib
import os
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path

print("\n" + "="*70)
print("BONUS PHASE: EXTRACTING REAL BLOCKCHAIN DATA FROM .DAT FILES")
print("="*70)

print("\nParser functions for Bitcoin .dat files...")

def parse_varint(data, offset):
    first_byte = data[offset]
    if first_byte < 0xfd:
        return first_byte, offset + 1
    elif first_byte == 0xfd:
        return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
    elif first_byte == 0xfe:
        return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5
    elif first_byte == 0xff:
        return struct.unpack('<Q', data[offset+1:offset+9])[0], offset + 9

def parse_transactions_from_dat(dat_file_path):
    transactions = []
    try:
        with open(dat_file_path, 'rb') as f:
            data = f.read()
        
        offset = 0
        tx_count = 0
        
        while offset < len(data) - 80:
            if data[offset:offset+4] == b'\xf9\xbe\xb4\xd9':
                offset += 4
                block_size = struct.unpack('<I', data[offset:offset+4])[0]
                offset += 4
                
                try:
                    version = struct.unpack('<I', data[offset:offset+4])[0]
                    offset += 4
                    offset += 32
                    offset += 32
                    block_timestamp = struct.unpack('<I', data[offset:offset+4])[0]
                    offset += 4
                    offset += 4
                    offset += 4
                    
                    tx_count_in_block, offset = parse_varint(data, offset)
                    
                    for tx_idx in range(tx_count_in_block):
                        tx_version = struct.unpack('<I', data[offset:offset+4])[0]
                        offset += 4
                        
                        input_count, offset = parse_varint(data, offset)
                        
                        for inp_idx in range(input_count):
                            offset += 32
                            offset += 4
                            script_len, offset = parse_varint(data, offset)
                            offset += script_len
                            offset += 4
                        
                        output_count, offset = parse_varint(data, offset)
                        
                        total_value = 0
                        for out_idx in range(output_count):
                            value_satoshi = struct.unpack('<Q', data[offset:offset+8])[0]
                            total_value += value_satoshi
                            offset += 8
                            script_len, offset = parse_varint(data, offset)
                            offset += script_len
                        
                        offset += 4
                        
                        tx = {
                            'timestamp': block_timestamp,
                            'input_count': input_count,
                            'output_count': output_count,
                            'value_satoshi': total_value
                        }
                        transactions.append(tx)
                        tx_count += 1
                        
                        if (tx_count % 10000) == 0:
                            print(f"  {tx_count} transactions parsed")
                
                except Exception as e:
                    break
            else:
                offset += 1
    
    except Exception as e:
        print(f"  Error reading file: {str(e)[:50]}")
    
    return transactions

print("\nProcessing .dat files...")
dat_folder = Path("data/blocks")
all_transactions = []

for dat_file in sorted(dat_folder.glob("blk*.dat"))[:8]:
    print(f"\n  Processing {dat_file.name}...")
    txs = parse_transactions_from_dat(dat_file)
    all_transactions.extend(txs)
    print(f"    {len(txs)} transactions extracted")

print(f"\nTotal: {len(all_transactions)} transactions extracted")

if len(all_transactions) > 0:
    df_txs = pd.DataFrame(all_transactions)
    
    df_txs['datetime'] = pd.to_datetime(df_txs['timestamp'], unit='s', utc=True)
    df_txs['hour'] = df_txs['datetime'].dt.floor('h')
    
    df_hourly_real = df_txs.groupby('hour').agg({
        'timestamp': 'count',
        'input_count': ['mean', 'sum'],
        'output_count': ['mean', 'sum'],
        'value_satoshi': ['sum', 'mean', 'std']
    }).reset_index()
    
    df_hourly_real.columns = [
        'timestamp', 'tx_count', 'avg_inputs', 'total_inputs',
        'avg_outputs', 'total_outputs', 'total_value_satoshi', 
        'avg_value_satoshi', 'std_value_satoshi'
    ]
    
    for col in ['total_value_satoshi', 'avg_value_satoshi', 'std_value_satoshi']:
        df_hourly_real[col] = df_hourly_real[col] / 100_000_000
    
    df_hourly_real = df_hourly_real.rename(columns={
        'total_value_satoshi': 'total_volume_btc',
        'avg_value_satoshi': 'avg_amount_btc',
        'std_value_satoshi': 'std_amount_btc'
    })
    
    print(f"\nAggregated to {len(df_hourly_real)} hours")
    print(f"\nBlockchain data preview:")
    print(df_hourly_real.head(10))
    
    print(f"\nStatistics:")
    print(f"  - tx_count: min={df_hourly_real['tx_count'].min():.0f}, max={df_hourly_real['tx_count'].max():.0f}, mean={df_hourly_real['tx_count'].mean():.0f}")
    print(f"  - total_volume_btc: min={df_hourly_real['total_volume_btc'].min():.2f}, max={df_hourly_real['total_volume_btc'].max():.2f}")
    print(f"  - avg_inputs: min={df_hourly_real['avg_inputs'].min():.1f}, max={df_hourly_real['avg_inputs'].max():.1f}")
    
    os.makedirs("data/parquet", exist_ok=True)
    blockchain_real_parquet = "data/parquet/blockchain_real_hourly.parquet"
    df_hourly_real['timestamp'] = df_hourly_real['timestamp'].astype(str)
    df_hourly_real.to_parquet(blockchain_real_parquet)
    print(f"\nSaved to: {blockchain_real_parquet}")
else:
    print("No transactions found in .dat files")


BONUS PHASE: EXTRACTING REAL BLOCKCHAIN DATA FROM .DAT FILES

Parser functions for Bitcoin .dat files...

Processing .dat files...

  Processing blk00013.dat...
  10000 transactions parsed
  20000 transactions parsed
  30000 transactions parsed
  40000 transactions parsed
  50000 transactions parsed
  60000 transactions parsed
  70000 transactions parsed
  80000 transactions parsed
  90000 transactions parsed
  100000 transactions parsed
  110000 transactions parsed
  120000 transactions parsed
  130000 transactions parsed
  140000 transactions parsed
  150000 transactions parsed
  160000 transactions parsed
  170000 transactions parsed
  180000 transactions parsed
  190000 transactions parsed
  200000 transactions parsed
  210000 transactions parsed
  220000 transactions parsed
  230000 transactions parsed
  240000 transactions parsed
  250000 transactions parsed
  260000 transactions parsed
  270000 transactions parsed
  280000 transactions parsed
  290000 transactions parsed
    29

In [ ]:
# Empty cell for organization

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, Imputer
from pyspark.sql.functions import col, year, expr, randn, abs as spark_abs

print('\n' + '='*70)
print('PHASE 4: ML DATA PREPARATION')
print('='*70)

feature_cols = [
    'hourly_return', 'hl_range', 'close_ma_24h', 'close_ma_7d', 'volatility_24h', 'volume_ratio',
    'tx_count', 'tx_count_ratio', 'total_volume_btc', 'volume_btc_ratio',
    'avg_inputs_log', 'avg_outputs_log', 'price_to_onchain', 'io_ratio'
]

print(f'Features: {len(feature_cols)} total')
df_features_filtered = df_features.filter(col('tx_count').isNotNull())
print(f'Rows with blockchain: {df_features_filtered.count()}')

if df_features_filtered.count() > 0:
    df_ml = df_features_filtered
    print('Using REAL blockchain data')
else:
    print('NO OVERLAP - Using synthetic fallback')
    df_ml = df_features.filter(col('price_direction').isNotNull())
    df_ml = df_ml.withColumn('tx_count', spark_abs(expr('cast(randn(42) * 500 + 1000 as double)')))
    df_ml = df_ml.withColumn('tx_count_ratio', expr('tx_count / 1000.0'))
    df_ml = df_ml.withColumn('total_volume_btc', spark_abs(expr('randn(43) * 100 + 200')))
    df_ml = df_ml.withColumn('volume_btc_ratio', expr('total_volume_btc / 200.0'))
    df_ml = df_ml.withColumn('avg_inputs_log', expr('log1p(abs(randn(44) * 1 + 2))'))
    df_ml = df_ml.withColumn('avg_outputs_log', expr('log1p(abs(randn(45) * 1 + 2))'))
    df_ml = df_ml.withColumn('price_to_onchain', expr('Close / (total_volume_btc + 1)'))
    df_ml = df_ml.withColumn('io_ratio', expr('exp(avg_inputs_log) / (exp(avg_outputs_log) + 0.1)'))

imputer = Imputer(inputCols=feature_cols, outputCols=feature_cols, strategy='median')
df_ml = imputer.fit(df_ml).transform(df_ml)
df_ml = df_ml.filter(col('price_direction').isNotNull())

assembler = VectorAssembler(inputCols=feature_cols, outputCol='features_raw', handleInvalid='skip')
df_ml = assembler.transform(df_ml)

scaler = StandardScaler(inputCol='features_raw', outputCol='features', withMean=True, withStd=True)
df_ml = scaler.fit(df_ml).transform(df_ml)

df_ml = df_ml.withColumn('year', year('Open time'))
df_train = df_ml.filter(col('year') < 2024)
df_test = df_ml.filter(col('year') >= 2024)

print(f'Train: {df_train.count()} rows (2018-2023)')
print(f'Test: {df_test.count()} rows (2024-2025)')
print('PHASE 4 COMPLETE!')

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

print('\n' + '='*70)
print('PHASE 5: MODEL TRAINING')
print('='*70)

lr = LogisticRegression(labelCol='price_direction', featuresCol='features', maxIter=100, regParam=0.01)
model_lr = lr.fit(df_train)
pred_lr = model_lr.transform(df_test)

rf = RandomForestClassifier(labelCol='price_direction', featuresCol='features', numTrees=50, maxDepth=10, seed=42)
model_rf = rf.fit(df_train)
pred_rf = model_rf.transform(df_test)

evaluator_auc = BinaryClassificationEvaluator(labelCol='price_direction', rawPredictionCol='rawPrediction', metricName='areaUnderROC')
evaluator_acc = MulticlassClassificationEvaluator(labelCol='price_direction', predictionCol='prediction', metricName='accuracy')

auc_lr = evaluator_auc.evaluate(pred_lr)
acc_lr = evaluator_acc.evaluate(pred_lr)
auc_rf = evaluator_auc.evaluate(pred_rf)
acc_rf = evaluator_acc.evaluate(pred_rf)

print(f'\nLogistic Regression - AUC: {auc_lr:.4f}, Accuracy: {acc_lr:.4f}')
print(f'Random Forest - AUC: {auc_rf:.4f}, Accuracy: {acc_rf:.4f}')
print(f'\nBest Model: {"Random Forest" if auc_rf > auc_lr else "Logistic Regression"} (AUC: {max(auc_rf, auc_lr):.4f})')